# Bonus Assignment 1

This is a Bonus Assignment try to complete all other Assignments before you reach here 

Try First Attempting Question 2 and then Question 1 


# Question 1
Solve this question on Kaggle! 
Download the notebook and submit it separately along with the other questions.

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

Use everything that you have learnt so far.

Analyse the data. Drop, scale or normalize features. Merge features if they correlated to make some meaningful features. 

Try out all the regression models. Use cross validation to find best parameters for each model.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

data = pd.read_csv(r'C:\Users\chand\summer_proj_back files\Week_1_submission\Bonus Assignment\train.csv')
data_train = pd.read_csv(r'C:\Users\chand\summer_proj_back files\Week_1_submission\Bonus Assignment\train.csv')

X = pd.DataFrame(data)
df = pd.DataFrame(data_train)

col_num = X.select_dtypes(include=['int','float']).columns.to_list()
col_cat = X.select_dtypes(include=['object']).columns.to_list()

df_col_num = df.select_dtypes(include=['int','float']).columns.to_list()
df_col_cat = df.select_dtypes(include=['object']).columns.to_list()

X_num = X[col_num]
X_cat = X[col_cat]

df_num = df[df_col_num]
df_cat = df[df_col_cat]

label = LabelEncoder()

for col in X_cat.columns:
    X_cat.loc[:, col] = label.fit_transform(X_cat[col])

for col in df_cat.columns:
    df_cat.loc[:, col] = label.fit_transform(df_cat[col])

X = pd.concat([X_num, X_cat], axis=1)
df = pd.concat([df_num, df_cat], axis=1)

# 'SalePrice' is the column to be predicted 
for col in X.columns:
    if X[col].isnull().any():
        X[col] = X[col].fillna(X[col].median())

for col in df.columns:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

correlation_matrix = X.corr()
print("Correlation Matrix:\n")
print(correlation_matrix['SalePrice'].sort_values(ascending=False))

# Feature scaling
x_train = X.drop('SalePrice', axis=1)
y_train = X['SalePrice']
x_test = df.drop('SalePrice', axis=1)
y_test = df['SalePrice']

col = x_train.columns.to_list()
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
x_train = pd.DataFrame(x_train, columns=col)
x_test = pd.DataFrame(x_test, columns=col)

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
}
param_grids = {
    'Linear Regression': {},
    'Ridge Regression': {'alpha': [0.1, 1.0, 10.0]}, 
    'Lasso Regression': {'alpha': [0.1, 1.0, 10.0]},  
    'Random Forest': {'n_estimators': [100, 200, 300]},  
    'Gradient Boosting': {'n_estimators': [100, 200, 300]},
}

for name, model in models.items():
    param_grid = param_grids[name]
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)
    best_parameter = grid_search.best_params_
    y_pred = grid_search.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"{name} MSE: {mse}")
    print(f"{name} the best parameters:{best_parameter}")
    print(f"{name} Predictions:{y_pred}")



Correlation Matrix:

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
                  ...   
FireplaceQu    -0.459605
GarageFinish   -0.549247
KitchenQual    -0.589189
BsmtQual       -0.620886
ExterQual      -0.636884
Name: SalePrice, Length: 81, dtype: float64
Linear Regression MSE: 920190718.9116274
Linear Regression the best parameters:{}
Linear Regression Predictions:[214672.60656409 201489.72427056 218396.08750263 ... 247066.24589047
 155610.24477255 162193.19871499]
Ridge Regression MSE: 1019326609.4753181
Ridge Regression the best parameters:{'alpha': 10.0}
Ridge Regression Predictions:[216114.70515544 201865.47973309 227194.58827161 ... 241763.76629659
 147397.64633457 154982.86394882]
Lasso Regression MSE: 912490162.52793
Lasso Regression the best parameters:{'alpha': 10.0}
Lasso Regression Predictions:[215295.34657705 202519.5400721  218610.16165355 ... 244547.87710363
 148090.51537274 158105.776405

# Question 2 : Regression using SVM and Tree Algorithms

In this question, we will be using the **insurance.csv** file which contain information on insurance charges based on the following informations: age,sex,bmi,region,number of children and whether the person is a smoker or not. You need to predict the charges based on the information given.

### 1. Load the data.

In [2]:
import pandas as pd
data = pd.read_csv(r'C:\Users\chand\summer_proj_back files\Week_1_submission\Bonus Assignment\insurance.csv')
df = pd.DataFrame(data)
print(df)

      age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]


### 2. Separate the numerical and categorical columns.
### 3. Label Encode the categorical columns.
### 4. Scale the numerical columns. (Scale the charges separately so that you can calculate errors afterwards.)

In [3]:
df_numerical = df[['age','bmi','children','charges']]
df_cat = df[['sex','smoker','region']]
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_numerical = scaler.fit_transform(df_numerical)
df_numerical = pd.DataFrame(df_numerical,columns=['age','bmi','children','charges'])
lalen = LabelEncoder()
for col in df_cat.columns:
    
    df_cat.loc[:,col] = lalen.fit_transform(df_cat[col])
print(df_numerical)
print(df_cat)
df_m = pd.concat([df_numerical,df_cat],axis = 1)
print(df_m.head())
col = df_m.columns[3]

X= df_m.drop(col,axis=1)
y= df_m[col]


           age       bmi  children   charges
0     0.021739  0.321227       0.0  0.251611
1     0.000000  0.479150       0.2  0.009636
2     0.217391  0.458434       0.6  0.053115
3     0.326087  0.181464       0.0  0.333010
4     0.304348  0.347592       0.0  0.043816
...        ...       ...       ...       ...
1333  0.695652  0.403820       0.6  0.151299
1334  0.000000  0.429379       0.0  0.017305
1335  0.000000  0.562012       0.0  0.008108
1336  0.065217  0.264730       0.0  0.014144
1337  0.934783  0.352704       0.0  0.447249

[1338 rows x 4 columns]
     sex smoker region
0      0      1      3
1      1      0      2
2      1      0      2
3      1      0      1
4      1      0      1
...   ..    ...    ...
1333   1      0      1
1334   0      0      0
1335   0      0      2
1336   0      0      3
1337   0      1      1

[1338 rows x 3 columns]
        age       bmi  children   charges sex smoker region
0  0.021739  0.321227       0.0  0.251611   0      1      3
1  0.000000  0

### 5. Split the data.

### 6. Support Vector Regressor

Here , you will use the SVR model from sklearn.svm and fit it on the training data. Then predict on the test data and calaculate MAE, MSE. But...

The SVR class contains many hyperparameters, example : kernel can have the following values : linear, rbf, poly, sigmoid.

Use **RandomizedSearchCV** from sklearn.model_selection , create a dictionary with keys 'kernel' and 'gamma' . As values of the keys, create a list of some possible values. Run a 3-fold cross validation test (cv=3) and find the best parameters. Then initiate the SVR model with those parameters.

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)


In [5]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error

svr= SVR(gamma='')
param_grid = {
    'kernel' : ['linear','poly','sigmoid','rbf'],
    'gamma' : ['auto','scale',1,0.1,2,0.01]
}
random_search = RandomizedSearchCV(svr,param_grid,cv=3,scoring='neg_mean_squared_error',random_state=42)
random_search.fit(X,y)
best_para = random_search.best_params_
print(best_para)
best_svr = SVR(gamma=best_para['gamma'],kernel=best_para['kernel'])
best_svr.fit(x_train,y_train)
y_predicted = best_svr.predict(x_test)
mae = mean_absolute_error(y_test,y_predicted)
mse = mean_squared_error(y_test,y_predicted)
print(f"mean_absolute_error{mae},mean_squared_error{mse}")



{'kernel': 'poly', 'gamma': 1}
mean_absolute_error0.08427233568372929,mean_squared_error0.008712015849188008
